
## The Battle of Neighborhoods

###  Course: Applied Data Science Capstone  


This notebook would contain Explaination and code for the coursera Capstone Project.

### Adding Required Libraries to create required Environment

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

pd.set_option('display.expand_frame_repr', False)

In [3]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


# Segmenting and Clustering Neighborhoods in Toronto

## 1. Data scraping

#### Description: 
Code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

In [4]:
# define the dataframe columns
column_names = ['Postal_Code','Borough', 'Neighborhood'] 

# instantiate the dataframe
nbr = pd.DataFrame(columns=column_names)
nbr

,Postal_Code,Borough,Neighborhood


#### Web scraping

##### Importing Data From provided URL

In [5]:
#import the library used to query a website
from urllib.request import urlopen

#specify the url
wiki = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

#Query the website and return the wiki to the variable 'page'
page = urlopen(wiki)

#import the Beautiful soup functions to parse the data returned from the website
from bs4 import BeautifulSoup

#Parse the html in the 'page' variable, and store it in Beautiful Soup format
soup = BeautifulSoup(page, "lxml")

In [15]:
# Uncomment below to Print HTML of the Page
#print(soup.prettify()) 

#Get table 
right_table=soup.find('table', class_='wikitable sortable')

right_table

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>
<tr>
<td>M6A</td>

##### Extract the information to DataFrame: 

we iterate through each row (tr) and then assign each element of tr (td) to a variable and append it to a list.

In [7]:
#Generate lists
A=[]
B=[]
C=[]

for row in right_table.findAll("tr"):
    cells = row.findAll('td')
    if len(cells)==3: #Only extract table body not heading
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

        
#Adding Data to our DataFrame
nbr['Postal_Code']=A
nbr['Borough']=B
nbr['Neighborhood']=C

#uncomment below to view dataframe nbr which contains data as retreived from the webpage in wiki
#nbr


####  Ignore cells with a borough that is Not assigned.

1. Compare with Each row if Borough is Not Assigned drop row.
2. Reset Index.

In [8]:
nbr = nbr.drop(nbr[nbr['Borough'].str.contains("Not assigned")==True].index, axis=0, inplace=False)

#Reset Index
nbr.index = pd.RangeIndex(len(nbr.index))


#### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

instantiate additional dataframe nbrNA.
Compare with Each row if Neighborhood is Not Assigned add value from Borough.

In [9]:
# instantiate additional dataframe nbrNA
column_n = ['NBRVAL'] 
nbrNA = pd.DataFrame(columns=column_n)
D=['Not assigned'] 
nbrNA['NBRVAL'] = D

#Dataframe contains 'Not assigned' value used for Comparison.
#print(nbrNA)

nbr1=nbr
#nbr1


#Compare with Each row if Neighborhood is Not Assigned add value from Borough.
for row_index,row in nbr.iterrows():
    if((nbr.loc[row_index,['Neighborhood']].values.astype('str') == 'Not assigned') or (nbr.loc[row_index,['Neighborhood']].values.astype('str') == 'Not assigned\n')):
        #print('inside if')
        nbr1.loc[row_index,['Neighborhood']] = nbr1.loc[row_index,['Borough']].values.astype('str') 
        
#Reset Index
#nbr.index = pd.RangeIndex(len(nbr.index))

#Uncomment below to view Dataframe nbr1 which contains Neighborhoods which were initially 'Not Assigned' now containing Borough values
print('nbr1')
nbr1

nbr1


,Postal_Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


##### More than one neighborhood can exist in one postal code area.

In [10]:
nbr2=nbr1.groupby('Postal_Code').agg({'Borough':'first',
                               'Neighborhood': ', '.join}).reset_index()

nbr2

,Postal_Code,Neighborhood,Borough
0,M1B,"Rouge, Malvern",Scarborough
1,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough
2,M1E,"Guildwood\n, Morningside, West Hill",Scarborough
3,M1G,Woburn,Scarborough
4,M1H,Cedarbrae\n,Scarborough
5,M1J,Scarborough Village,Scarborough
6,M1K,"East Birchmount Park\n, Ionview, Kennedy Park",Scarborough
7,M1L,"Clairlea, Golden Mile, Oakridge",Scarborough
8,M1M,"Cliffcrest, Cliffside, Scarborough Village West\n",Scarborough
9,M1N,"Birch Cliff, Cliffside West\n",Scarborough


In [11]:
print(nbr2.shape)
print('nbr2')
nbr2


(103, 3)
nbr2


,Postal_Code,Neighborhood,Borough
0,M1B,"Rouge, Malvern",Scarborough
1,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough
2,M1E,"Guildwood\n, Morningside, West Hill",Scarborough
3,M1G,Woburn,Scarborough
4,M1H,Cedarbrae\n,Scarborough
5,M1J,Scarborough Village,Scarborough
6,M1K,"East Birchmount Park\n, Ionview, Kennedy Park",Scarborough
7,M1L,"Clairlea, Golden Mile, Oakridge",Scarborough
8,M1M,"Cliffcrest, Cliffside, Scarborough Village West\n",Scarborough
9,M1N,"Birch Cliff, Cliffside West\n",Scarborough


## Filtering Neighborhood

Creating a copy of Dataframe. Dropping neighborhoods which do not Belong to Toronto

In [16]:

# define the dataframe columns
column_names = ['Postal_Code','Borough', 'Neighborhood'] 
# instantiate the dataframe
neighbor = pd.DataFrame(columns=column_names)

neighbor = nbr2.drop(nbr2[nbr2['Borough'].str.contains("Toronto")==False].index, axis=0, inplace=False)

#Reset Index
neighbor.index = pd.RangeIndex(len(neighbor.index))
print('neighbor')
print(neighbor)

neighbor
   Postal_Code                                       Neighborhood           Borough
0          M4E                                        The Beaches      East Toronto
1          M4K                     The Danforth West\n, Riverdale      East Toronto
2          M4L                   The Beaches West\n, India Bazaar      East Toronto
3          M4M                                  Studio District\n      East Toronto
4          M4N                                      Lawrence Park   Central Toronto
5          M4P                                 Davisville North\n   Central Toronto
6          M4R                               North Toronto West\n   Central Toronto
7          M4S                                       Davisville\n   Central Toronto
8          M4T                      Moore Park, Summerhill East\n   Central Toronto
9          M4V  Deer Park, Forest Hill SE\n, Rathnelly, South ...   Central Toronto
10         M4W                                           Rosedale  

In [17]:
#Keeping nbr_ungrp for ease of location calculations, using nbr1 by dropping Non Toronto Boroughs.
# define the dataframe columns
column_names = ['Postal_Code','Borough', 'Neighborhood'] 
# instantiate the dataframe
nbr_ungrp = pd.DataFrame(columns=column_names)

nbr_ungrp = nbr1.drop(nbr1[nbr1['Borough'].str.contains("Toronto")==False].index, axis=0, inplace=False)

#Reset Index
nbr_ungrp.index = pd.RangeIndex(len(nbr_ungrp.index))

#Uncomment below to view Dataframe
print(nbr_ungrp)

   Postal_Code           Borough                                       Neighborhood
0          M5A  Downtown Toronto                                       Harbourfront
1          M5A  Downtown Toronto                                        Regent Park
2          M5B  Downtown Toronto                                           Ryerson

3          M5B  Downtown Toronto                                   Garden District

4          M5C  Downtown Toronto                                     St. James Town
5          M4E      East Toronto                                        The Beaches
6          M5E  Downtown Toronto                                        Berczy Park
7          M5G  Downtown Toronto                                Central Bay Street

8          M6G  Downtown Toronto                                          Christie

9          M5H  Downtown Toronto                                          Adelaide

10         M5H  Downtown Toronto                                            

## Capturing Longitude And Latitude info of Locations

using the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html

In [18]:
!conda install -c conda-forge geopy --yes 
# convert an address into latitude and longitude values
import time
from geopy.geocoders import Nominatim

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

   

### Looping through locations to fetch Latitude and Longitude.

In [69]:
geolocator = Nominatim(user_agent="my_app app123")
#print('here')

for row_index, item in nbr_ungrp.iterrows():
    
    list1 = nbr_ungrp.loc[[row_index],['Neighborhood']].values.astype('str')
    loc = ' , Toronto, Ontario, Canada'
    list1.astype('str')
    list1 = np.append(list1, loc)
    latitude = None
    longitude = None
    location = None
    
    location = geolocator.geocode(list1 , limit = 15)
    #print(loc)
    #print(row_index)
    time.sleep(5)
    if(location is not None):
        #print(location)
        #print("location Address : ")
        #print(location.address)
        #print('latitude, longitude : ')
        #print((location.latitude, location.longitude))
        nbr_ungrp.loc[nbr_ungrp.index[row_index], 'Latitude'] = location.latitude
        nbr_ungrp.loc[nbr_ungrp.index[row_index], 'Longitude'] = location.longitude
        #print(nbr_ungrp[row_index])    

print(nbr_ungrp)

   Postal_Code           Borough                                       Neighborhood   Latitude  Longitude
0          M5A  Downtown Toronto                                       Harbourfront  43.640080 -79.380150
1          M5A  Downtown Toronto                                        Regent Park  43.660706 -79.360457
2          M5B  Downtown Toronto                                           Ryerson
  43.657474 -79.377905
3          M5B  Downtown Toronto                                   Garden District
  43.656502 -79.377128
4          M5C  Downtown Toronto                                     St. James Town  43.669403 -79.372704
5          M4E      East Toronto                                        The Beaches  43.671024 -79.296712
6          M5E  Downtown Toronto                                        Berczy Park  43.647989 -79.375393
7          M5G  Downtown Toronto                                Central Bay Street
        NaN        NaN
8          M6G  Downtown Toronto              

# Neighborhood analysis


## Importing Libraries for Foursqaue to analyse Neighborhood.

In [70]:

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Importing to use the Foursquare API lab
!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Foursquare and map plotting Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         560 KB

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge


folium-0.5.0         | 45 KB     | #############################

#### Number of Boroughs and Neighborhoods

In [71]:

print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(nbr_ungrp['Borough'].unique()),
        nbr_ungrp.shape[0]
    )
)

#Dropping NANs
nbr_ungrp.dropna(inplace =True)
#Reset Index
nbr_ungrp.index = pd.RangeIndex(len(nbr_ungrp.index))
nbr_ungrp

The dataframe has 4 boroughs and 74 neighborhoods.


,Postal_Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.640080,-79.380150
1,M5A,Downtown Toronto,Regent Park,43.660706,-79.360457
2,M5B,Downtown Toronto,Ryerson,43.657474,-79.377905
3,M5B,Downtown Toronto,Garden District,43.656502,-79.377128
4,M5C,Downtown Toronto,St. James Town,43.669403,-79.372704
5,M4E,East Toronto,The Beaches,43.671024,-79.296712
6,M5E,Downtown Toronto,Berczy Park,43.647989,-79.375393
7,M6G,Downtown Toronto,Christie,43.664111,-79.418405
8,M5H,Downtown Toronto,Adelaide,43.651923,-79.370879
9,M5H,Downtown Toronto,King,43.648949,-79.377754


#### Using geopy library to get the latitude and longitude values of Toronto City.

In [72]:
address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="my-app123")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


## PART 3:

##### Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.



In [73]:
#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

#Use geopy library to get the latitude and longitude values of Toronto
address = 'Toronto,ON'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of ',address,' are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of  Toronto,ON  are 43.653963, -79.387207.


/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:7: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


In [76]:
#!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(nbr_ungrp['Latitude'], nbr_ungrp['Longitude'], nbr_ungrp['Borough'], nbr_ungrp['Neighborhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
            
map_toronto